In [2]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib as plt
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("Tensorflow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly))

Tensorflow version: 1.8.0
Eager execution: <function executing_eagerly at 0x1060fdae8>


Download data

In [6]:
## Download iris training data
train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname="iris_training.csv",
                                           origin=train_dataset_url)

print("Local copy of dataset: {}".format(train_dataset_fp))

Local copy of dataset: /Users/ni41435_ca/.keras/datasets/iris_training.csv


In [7]:
!head -n5 {train_dataset_fp}

120,4,setosa,versicolor,virginica
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0


In [8]:
def parse_csv(line):
  example_defaults = [[0.], [0.], [0.], [0.], [0]]  # sets field types
  parsed_line = tf.decode_csv(line, example_defaults)
  # First 4 fields are features, combine into single tensor
  features = tf.reshape(parsed_line[:-1], shape=(4,))
  # Last field is the label
  label = tf.reshape(parsed_line[-1], shape=())
  return features, label

In [9]:
help(tf.decode_csv)

Help on function decode_csv in module tensorflow.python.ops.parsing_ops:

decode_csv(records, record_defaults, field_delim=',', use_quote_delim=True, name=None, na_value='')
    Convert CSV records to tensors. Each column maps to one tensor.
    
    RFC 4180 format is expected for the CSV records.
    (https://tools.ietf.org/html/rfc4180)
    Note that we allow leading and trailing spaces with int or float field.
    
    Args:
      records: A `Tensor` of type `string`.
        Each string is a record/row in the csv and all records should have
        the same format.
      record_defaults: A list of `Tensor` objects with specific types.
        Acceptable types are `float32`, `float64`, `int32`, `int64`, `string`.
        One tensor per column of the input record, with either a
        scalar default value for that column or empty if the column is required.
      field_delim: An optional `string`. Defaults to `","`.
        char delimiter to separate fields in a record.
      use_qu

In [10]:
train_dataset = tf.data.TextLineDataset(train_dataset_fp)
train_dataset = train_dataset.skip(1)             # skip the first header row
train_dataset = train_dataset.map(parse_csv)      # parse each row
train_dataset = train_dataset.shuffle(buffer_size=1000)  # randomize
train_dataset = train_dataset.batch(32)

## View a single entry from batch
features, label = iter(train_dataset).next()
print("example features:", features[0])
print("example label:", label[0])

example features: tf.Tensor([ 6.4000001   2.79999995  5.5999999   2.0999999 ], shape=(4,), dtype=float32)
example label: tf.Tensor(2, shape=(), dtype=int32)


In [11]:
train_dataset

<BatchDataset shapes: ((?, 4), (?,)), types: (tf.float32, tf.int32)>

# Creating a model using Keras

Predict label(setosa, versicolor, virginica) using features(sepel length, width, petal length, width)

In [13]:
# tf.keras.Sequential is a linear stack of layers. 
# Constructor takes a list of layer instances, in this case, 
# two "Dense" layers with 10 nodes each, and an output layer of 3 nodes of our 
# label prediction.
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation="relu", input_shape=(4,)),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(3)
])

In [14]:
model

In [15]:
def loss(model, x, y):
    y_ = model(x)
    return tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)

In [16]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, model.variables)

In [17]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)


In [18]:
optimizer